# Brainstorm

- Task 1: Load search filter from https://docs.google.com/spreadsheets/d/1YvmsBmfjYxYi-aYJE2EX0566vyhxqMTwdir8zUtK-ls/edit#gid=2030777624

- Task 2: Search for the lessons

- Task 3: Get the content on QLink
    - Task 3.1: Go through each page and copy paste the content to a doc, stored as dictionary
    - Task 3.2: Click PDF, download PDF, load PDF content to the dictionary
    
- Task 4: Load the content to Grammarly
    - Task 4.1:
    
- Task 5: Report the content to a csv
    - Task 5.1: Click plagiarism
    - Task 5.1: Identify text 

In [3]:
# Import libraries and packages for the project 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
from time import sleep
import csv
print('- Finish importing packages')

# setup selenium webdriver
opt = webdriver.ChromeOptions()
#opt.add_extension("Block-image_v1.1.crx")
opt.add_argument('--disable-gpu')
opt.add_argument("--window-size=1920,1080")
opt.add_argument("--start-maximized")
opt.add_argument('--disable-dev-shm-usage')
opt.add_argument('--no-sandbox')
opt.add_argument('--ignore-certificate-errors')
#opt.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36")
driver = webdriver.Chrome(options=opt)

# setup wait
wait = WebDriverWait(driver, 10)

# Task 2
# Task 2.1: Open Chrome and Access login site
driver = webdriver.Chrome()
sleep(2)
url = input('Please input URL: ')
driver.get(url)
print('- Finish initializing a driver')
sleep(2)

- Finish importing packages
Please input URL: https://link.quipper.com/en/organizations/58b75e136e392c6f6700000a/curriculum#curriculum
- Finish initializing a driver


In [10]:
# Task 1.2: Key in login credentials
email_field = driver.find_element_by_name('email_or_username')
email_field.send_keys('daphne.huynh@quipper.com')
print('- Finish keying in email')
sleep(3)

password_field = driver.find_element_by_xpath('//*[@id="mount"]/div[1]/div[2]/div/div[2]/form/div[2]/div[1]/div[2]/div/input')
password_field.send_keys('Daphne@Quipper123')
print('- Finish keying in pw')
sleep(3)

password_field = driver.find_element_by_xpath('//*[@id="mount"]/div[1]/div[2]/div/div[2]/form/div[2]/div[1]/div[2]/div/input')
password_field.send_keys(Keys.RETURN)
print('- Finish Task 1: Login')
sleep(3)

- Finish keying in email
- Finish keying in pw
- Finish Task 1: Login to Linkedin


In [15]:
url = input('Please input URL: ')
driver.get(url)
print('- Finish initializing a driver')
sleep(2)

Please input URL: https://link.quipper.com/en/organizations/58b75e136e392c6f6700000a/curriculum#curriculum
- Finish initializing a driver


In [46]:
# Task 3: Get the content  
page_source = BeautifulSoup(driver.page_source)
all_page = page_source.find('div', class_ = 'stepper')
pages = all_page.find_all('a', class_='stepper-step')

for page_no in range(1, len(pages) + 1):
    page = driver.find_element_by_xpath(f'//*[@id="yield"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/a[{page_no}]')
    # page.click()
    driver.execute_script("arguments[0].click();", page)
    print(f'{page_no}: clicked')
    sleep(3)
    page_source = BeautifulSoup(driver.page_source)
    sleep(2)
    page_content = page_source.find('div', class_='question-content-lesson-chapter Chapter--12HKG').get_text()
    print(page_content, '\n\n')

1: clicked
Objective
At the end of this lesson, you should be able to use appropriate modifiers.

Essential Question
How do modifiers affect cohesion in writing?
Report errors 


2: clicked
Learn about It!
A dangling modifier is a word or group of words that describes something that is missing in a sentence, such as a noun or a pronoun. Therefore, it is a modifier that has nothing to describe or modify. 
Report errors 


3: clicked
Learn about It!
Correcting a Dangling Modifier
Dangling modifiers make the meaning of a sentence unclear and sometimes illogical. This type of error occurs because of a failure to indicate the word that the modifier should be describing. 
To correct a dangling modifier, the writer must supply the missing modified word to complete the thought of the sentence. Take note that the supplied modified word(s) should also be appropriate for the context of the sentence.
Report errors 


4: clicked
Example

Climbing a tree, my water jug fell on the ground.

In this ex